In [ ]:
import pandas as pd
import ast
import numpy as np

# --- 1. CARREGAR E PREPARAR O DATAFRAME ---
# 'low_memory=False' é usado para evitar problemas de tipo de dados mistos (DtypeWarning).
print("--- Carregando o Dataset ---")
try:
    df = pd.read_csv('archive/movies_metadata.csv', low_memory=False)
except FileNotFoundError:
    print("ERRO: Arquivo 'archive/movies_metadata.csv' não encontrado. Verifique o caminho.")
    exit()

# Definir todas as colunas JSON que potencialmente existem no arquivo
ALL_JSON_COLS = ['genres', 'belongs_to_collection', 'production_companies', 
                 'production_countries', 'spoken_languages', 'cast', 'crew']

# Lista final das colunas que realmente existem no seu DataFrame
JSON_COLS_TO_PROCESS = [col for col in ALL_JSON_COLS if col in df.columns]
print(f"Colunas JSON que serão processadas: {JSON_COLS_TO_PROCESS}")

# --- 2. FUNÇÃO DE DESSERIALIZAÇÃO SEGURA ---
def safe_literal_eval(val):
    """Converte strings literais (JSON/Python) em objetos Python, tratando nulos e erros."""
    if pd.isna(val) or val in ['', 'None']:
        return None
    try:
        # ast.literal_eval é mais seguro que eval() e trata strings literais de Python
        return ast.literal_eval(val)
    except:
        return None

# --- 3. APLICAR DESSERIALIZAÇÃO (CRIA AS COLUNAS '_obj') ---
print("--- Aplicando Desserialização ---")
for col in JSON_COLS_TO_PROCESS:
    # 3.1 Cria a coluna temporária de objeto (ex: df['genres_obj'])
    df[f'{col}_obj'] = df[col].apply(safe_literal_eval)
    
    # 3.2 Preenchimento de nulos para garantir que o objeto seja Lista ou Dicionário
    # (Para evitar erros nos passos de concatenação/contagem)
    if col == 'belongs_to_collection':
        # Deve ser Dicionário {} para filmes sem coleção
        df[f'{col}_obj'] = df[f'{col}_obj'].apply(lambda x: x if isinstance(x, dict) else {})
    elif isinstance(df[f'{col}_obj'].iloc[0], list) or col not in ['belongs_to_collection']: 
        # Deve ser Lista [] para colunas como genres, companies, etc.
        df[f'{col}_obj'] = df[f'{col}_obj'].apply(lambda x: x if isinstance(x, list) else [])


# --- 4. TRATAMENTO DA COLUNA 'belongs_to_collection' (Dicionário Único) ---

if 'belongs_to_collection' in JSON_COLS_TO_PROCESS:
    # Extrai o nome da coleção e coloca em uma coluna simples
    df['collection_name'] = df['belongs_to_collection_obj'].apply(
        lambda x: x.get('name') if isinstance(x, dict) else None
    )


# --- 5. NORMALIZAÇÃO DE LISTAS (Contagem e Lista Simples de Nomes) ---

# Colunas que são listas e que foram criadas no passo 3 (excluindo o dicionário 'belongs_to_collection')
LIST_COLS_OBJ = [col for col in JSON_COLS_TO_PROCESS if col != 'belongs_to_collection']

print("--- Criando Contagens e Listas Simples ---")

for col in LIST_COLS_OBJ:
    obj_col = f'{col}_obj' 
    
    # 5.1. Extrai o número de itens na lista (ex: genres_count)
    df[f'{col}_count'] = df[obj_col].apply(lambda x: len(x) if isinstance(x, list) else 0)
    
    # 5.2. Cria uma coluna de texto simples (ex: "Action|Comedy")
    def extract_names(lista):
        if isinstance(lista, list):
            # Extrai o nome de cada dicionário na lista (com tratamento seguro)
            names = [d.get('name') for d in lista if isinstance(d, dict) and d.get('name')]
            return '|'.join(names)
        return ''

    df[f'{col}_list'] = df[obj_col].apply(extract_names)


# --- 6. REMOÇÃO FINAL DAS COLUNAS JSON ORIGINAIS E OBJETOS TEMPORÁRIOS ---

# Colunas originais com JSON/Python object:
cols_to_drop = JSON_COLS_TO_PROCESS

# Colunas temporárias (Python objects) criadas na Etapa 3:
cols_to_drop.extend([f'{col}_obj' for col in JSON_COLS_TO_PROCESS])

# Remove as colunas complexas, preservando apenas as colunas limpas (e as contagens/listas)
df = df.drop(columns=cols_to_drop, errors='ignore')


# --- 7. EXIBIR O RESULTADO FINAL ---
print("\n" + "="*50)
print("✅ LIMPEZA E NORMALIZAÇÃO BÁSICA CONCLUÍDAS.")
print("="*50)

print("\nColunas Restantes no DataFrame Limpo:")
print(df.columns.tolist())

print("\nExemplo de Dados Limpos:")
# Exibe as novas colunas limpas: o nome da coleção e as contagens/listas
print(df[['title', 'collection_name', 'genres_count', 'genres_list', 'production_companies_count']].head())

In [ ]:
df_filtrado.to_csv('Metadados Filme.csv')

In [ ]:
df_filtrado = df

In [ ]:
df_filtrado.drop(columns=['homepage', 'imdb_id', 'budget', 'poster_path', 'collection_name', 'production_companies_count', 'production_companies_list', 'production_countries_count'], inplace=True)

In [ ]:
df_filtrado.rename(columns={'adult': 'Adulto', 'original_language': 'Idioma Original', 'original_title': 'Título Original', 'overview': "Descrição", 'popularity': 'Popularidade', 'release_date': 'Data de Lançamento', 'revenue': 'Receita', 'runtime': 'Duração', 'status': 'Status', 'production_countries_list': 'País de Origem', 'spoken_languages_count': 'Idiomas falados no filme', 'spoken_languages_list': 'Lista de Idiomas', 'title': 'Título em inglês'}, inplace=True)

In [ ]:
df_brazil = df_filtrado[df_filtrado['País de Origem'] == 'Brazil']

In [ ]:
df_brazil.sort_values('Data de Lançamento',ascending=False)

In [ ]:
df_filtrado[['id', 'Título Original', 'Título em inglês', 'País de Origem', 'Idioma Original', 'Idiomas falados no filme', 'Lista de Idiomas', 'genres_list', 'vote_count', 'vote_average', 'Popularidade', 'Duração', 'Data de Lançamento', 'Adulto', 'Status']]

In [ ]:
df_analise = df_filtrado[['Título Original', 'Título em inglês', 'País de Origem', 'Idioma Original']].copy() 

# 2. Cria a coluna booleana 'Analise'
# Verifica se os valores são iguais e armazena o resultado (True/False)
df_analise['Analise'] = df_filtrado['Título Original'] == df_filtrado['Título em inglês']

df_analise.loc[df_analise['Analise'] == False]

In [ ]:
df_analise

Analisando o DataSet Keywords